In [ ]:
"""
Author: Yuqiang (Ethan) Heng
"""
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.layers import Dense
from keras.models import Model
from keras.layers import Input
from keras import backend as K
from keras.utils import np_utils

def cart2sph(xyz,center):
    x = np.subtract(xyz[:,0],center[0])
    y = np.subtract(xyz[:,1],center[1])
    z = np.subtract(xyz[:,2],center[2])
    rtp = np.zeros(xyz.shape)
    r = np.sqrt(np.power(x,2)+np.power(y,2)+np.power(z,2))
    theta = np.arccos(np.divide(z,r))
    phi = np.arctan2(y,x)
    rtp[:,0] = r
    rtp[:,1] = theta
    rtp[:,2] = phi
    return rtp

In [ ]:
# Prepare data for training
loc_noise = 2.0409 # noise in the cartesian coordinates of UEs
static_dataset = np.load('./Dataset/MISO_BeamTraining_FineGrid_processed_polar.npy')
X_static = static_dataset[:,0:3]
Y = np.copy(static_dataset[:,3])
# convert to cartesian coordinates
# the cartesian coordinates of the BS is [641,435,10]
X_static_cart = np.copy(static_dataset[:,0:3])
cartesian_x = np.multiply(np.multiply(static_dataset[:,0],np.sin(static_dataset[:,1])),np.cos(static_dataset[:,2]))
cartesian_y = np.multiply(np.multiply(static_dataset[:,0],np.sin(static_dataset[:,1])),np.sin(static_dataset[:,2]))
cartesian_z = np.multiply(static_dataset[:,0],np.cos(static_dataset[:,1]))
X_static_cart[:,0] = cartesian_x + 641
X_static_cart[:,1] = cartesian_y + 435
X_static_cart[:,2] = cartesian_z + 10
static_dataset[:,0:3] = X_static_cart
# add noise
X = X_static_cart + np.random.normal(0,loc_noise,X_static_cart.shape)
X = cart2sph(X, [641,435,10])
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
nbeam = 64
nvalid_rx_static = X.shape[0]
bf_rx_per_beam = {i:[] for i in range(nbeam)}
for i in range(nvalid_rx_static):
    bf_rx_per_beam[int(Y[i])].append(i)
train_ratio = 0.8 # 0.8/0.1/0.1 train/val/test split
train_rx_idc = []
test_rx_idc = []
val_rx_idc = []
for i in range(nbeam):
    if len(bf_rx_per_beam[i]) >= 10:
        train_inst = np.random.choice(bf_rx_per_beam[i], int(np.floor(train_ratio*len(bf_rx_per_beam[i]))), replace = False)
        remain_idc = np.setdiff1d(bf_rx_per_beam[i], train_inst)
        val_inst = np.random.choice(remain_idc, int(np.floor(0.5*len(remain_idc))), replace = False)
        test_inst = np.setdiff1d(remain_idc, val_inst)
        for train_rx_idx in train_inst:
            train_rx_idc.append(train_rx_idx)
        for val_rx_idx in val_inst:
            val_rx_idc.append(val_rx_idx)
        for test_rx_idx in test_inst:
            test_rx_idc.append(test_rx_idx)
X_train = X[train_rx_idc,:]
Y_train = Y[train_rx_idc]
X_val = X[val_rx_idc,:]
Y_val = Y[val_rx_idc]
X_test = X[test_rx_idc,:]
Y_test = Y[test_rx_idc]
Y_train_onehot = np_utils.to_categorical(encoder.transform(Y_train))
Y_val_onehot = np_utils.to_categorical(encoder.transform(Y_val))
Y_test_onehot = np_utils.to_categorical(encoder.transform(Y_test))

In [ ]:
K.clear_session()
input_location = Input(shape=(3,))
dense1 = Dense(6, activation ='sigmoid')(input_location)
dense2 = Dense(18, activation = 'sigmoid')(dense1)
dense3 = Dense(48, activation = 'sigmoid')(dense2)
output = Dense(Y_train_onehot.shape[1], activation = 'softmax')(dense3)
model = Model(inputs = input_location, outputs = output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train_onehot, validation_data = (X_val, Y_val_onehot), epochs = 20, batch_size = 100, verbose=1)